v1 meta store product availability checker

In [4]:
##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

cleaned = []

# missing metadata:
# Meta Quest Touch Pro Controllers
# Meta Quest 3 Charging Dock
#
# missing 'with' makes scraping hard

# verified working
# product_list = ['quest 3 standard facial interface',
#                 'quest 3 carrying case',
#                 'quest 3 elite strap battery',
#                 'quest 2 elite strap battery',
#                 'quest pro carry case',
#                 'quest 2 logitech pro gaming headset']

# product_list = ['quest 3 carrying case']


for product in product_list:
    url_completion = product.replace(' ', '-')
    url = 'https://www.meta.com/quest/accessories/{}'.format(url_completion)

    # bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    text = soup.get_text().lower().replace('\n', '')

    # account for different product names in metadata
    # product is the url name
    # product_pattern is the product metadata name
    if product == 'quest 3 elite strap battery':
        product_pattern = 'quest 3 elite strap'
    elif product == 'quest 2 elite strap battery':
        product_pattern = 'quest 2 elite strap'
    elif product == 'quest pro carry case':
        product_pattern = 'quest pro carrying case'
    elif product == 'quest 2 logitech pro gaming headset':
        product_pattern = 'logitech g pro gaming headset for meta quest 2'
    else:
        product_pattern = product

    # print(product_pattern)
    pattern = '{} (.*?) usd'.format(re.escape(product_pattern))
    # print(pattern)
    # print(text)
    match = re.search(pattern, text)
    match_text = match.group()

    # column separator prep for pandas table
    replacement_key = product.split()[-1]
    if '$' in match_text:
        cleaned.append(match_text.replace('$', '`$').replace(
            replacement_key, replacement_key + '`'))
    print(cleaned)

print('done')
print(cleaned)

['quest 3 standard facial interface` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaq  add to bag`$34.99 usd']
['quest 3 standard facial interface` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaq  add to bag`$34.99 usd', 'quest 3 carrying case` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaqreviews  add to bag`$69.99 usd']
['quest 3 standard facial interface` | meta | meta store    free delivery        extended returns‡        warranty  skip to main contentmeta questray-ban | metaapps and gamesabout metasupportoverviewtech specsfaq  add to bag`$34.99 usd', 'quest 3 carrying case` | meta | meta store    free de

In [ ]:
##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

cleaned = []
url = 'https://www.meta.com/quest/accessories/quest-2-logitech-pro-gaming-headset/'

driver = webdriver.Chrome()  # bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
text = soup.get_text().lower()

print(text)